In [2]:
from os.path import isfile, isdir, splitext
import argparse
import pickle as cPickle
import numpy.random as npr
from heuristic_sentence_splitter import sent_tokenize_rules
from mimic_querier import *

In [3]:
args = {}
args['psql_host'] = 'localhost'
args['psql_dbname'] = 'mimiciv'
args['psql_schema_name'] = 'mimiciv_derived, mimiciv_hosp, mimiciv_icu, mimiciv_ed'
args['psql_user'] = 'postgres'
args['psql_password'] = 'root'
dbname = args['psql_dbname']
schema_name = args['psql_schema_name']
query_args = {'dbname': dbname}
if args['psql_host'] is not None: 
    query_args['host'] = args['psql_host']
if args['psql_user'] is not None: 
    query_args['user'] = args['psql_user']
if args['psql_password'] is not None: 
    query_args['password'] = args['psql_password']

In [9]:
querier = MIMIC_Querier(query_args=query_args, schema_name=schema_name)
querier.connect()

In [10]:
stay_id_count = pd.read_sql_query('select count(distinct stay_id) from mimiciv_derived.icustay_detail ',querier.connection ).iloc[0][0]

In [11]:
stay_id_count

73181

In [12]:
query = '''
with
t1 as (select itemid, chartevents.stay_id from mimiciv_icu.chartevents  inner join mimiciv_derived.icustay_detail on icustay_detail.stay_id = chartevents.stay_id
where chartevents.charttime between icustay_detail.icu_intime and icu_outtime
group by itemid, chartevents.stay_id)
, t2 as (select itemid, count(*) as count from t1 group by itemid order by count desc)
select t2.count, d_items.* from t2 inner join mimiciv_icu.d_items on t2.itemid = d_items.itemid order by count desc ;
'''
charevent_with_txt = pd.read_sql_query(query,querier.connection )

In [14]:
charevent_with_txt.insert(1,'percentage',charevent_with_txt['count'] / stay_id_count)

In [16]:
charevent_with_txt

,count,percentage,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,73163,0.999754,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
1,73083,0.998661,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
2,73069,0.998470,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
3,72755,0.994179,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,None,Text,NaN,NaN
4,72640,0.992607,220739,GCS - Eye Opening,Eye Opening,chartevents,Neurological,None,Text,NaN,NaN
5,72629,0.992457,223900,GCS - Verbal Response,Verbal Response,chartevents,Neurological,None,Text,NaN,NaN
6,72618,0.992307,223901,GCS - Motor Response,Motor Response,chartevents,Neurological,None,Text,NaN,NaN
7,72586,0.991869,224641,Alarms On,Alarms On,chartevents,Alarms,None,Checkbox,NaN,NaN
8,72558,0.991487,224080,Head of Bed,HOB,chartevents,Treatments,None,Text,NaN,NaN
9,72476,0.990366,224082,Turn,Turn,chartevents,Treatments,None,Text,NaN,NaN


In [18]:
charevent_with_txt.loc[charevent_with_txt['param_type'] == 'Numeric'].to_csv('charevent.csv', index = False)

In [20]:
charevent_with_txt.to_csv('charevent_with_txt.csv', index = False)

In [7]:
query = '''
with t1 as (select itemid, chartevents.stay_id, chartevents.value from mimiciv_icu.chartevents  inner join mimiciv_derived.icustay_detail on icustay_detail.stay_id = chartevents.stay_id
where chartevents.charttime between icustay_detail.icu_intime and icu_outtime),
t2 as (select * from mimiciv_icu.d_items where linksto = 'chartevents' and param_type = 'Text'),
t3 as (select t1.stay_id, t1.itemid, t1.value from t1 inner join t2 on t1.itemid = t2.itemid)
select itemid, count(distinct value) as type from t3 group by itemid order by type desc;

'''
labevent_txt_distanct = pd.read_sql_query(query,querier.connection )

In [8]:
labevent_txt_distanct

,itemid,type
0,220001,1045
1,228497,95
2,228496,95
3,228499,95
4,228498,95
5,228559,94
6,228500,92
7,228560,92
8,228506,89
9,228561,87


In [36]:
charevent_txt_distanct = charevent_with_txt.set_index('itemid').join(labevent_txt_distanct.set_index('itemid')).reset_index()

In [37]:
charevent_txt_distanct

,itemid,count,percentage,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue,type
0,220045,73163,0.999754,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN,NaN
1,220210,73083,0.998661,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN,NaN
2,220277,73069,0.998470,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN,NaN
3,220048,72755,0.994179,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,None,Text,NaN,NaN,26.0
4,220739,72640,0.992607,GCS - Eye Opening,Eye Opening,chartevents,Neurological,None,Text,NaN,NaN,4.0
5,223900,72629,0.992457,GCS - Verbal Response,Verbal Response,chartevents,Neurological,None,Text,NaN,NaN,6.0
6,223901,72618,0.992307,GCS - Motor Response,Motor Response,chartevents,Neurological,None,Text,NaN,NaN,6.0
7,224641,72586,0.991869,Alarms On,Alarms On,chartevents,Alarms,None,Checkbox,NaN,NaN,NaN
8,224080,72558,0.991487,Head of Bed,HOB,chartevents,Treatments,None,Text,NaN,NaN,8.0
9,224082,72476,0.990366,Turn,Turn,chartevents,Treatments,None,Text,NaN,NaN,9.0


In [39]:
charevent_txt_distanct.to_csv('charevent_txt_distanct.csv', index = False)

In [27]:
def get_variable_mapping(mimic_mapping_filename):
    # Read in the second level mapping of the itemids
    var_map = pd.read_csv(mimic_mapping_filename, index_col=None)
    var_map = var_map.ix[(var_map['LEVEL2'] != '') & (var_map['COUNT']>0)]
    var_map = var_map.ix[(var_map['STATUS'] == 'ready')]
    var_map['ITEMID'] = var_map['ITEMID'].astype(int)

    return var_map

In [31]:
mimic_mapping_filename = './resources/itemid_to_variable_map.csv'
var_map = get_variable_mapping(mimic_mapping_filename)

D:\software\Anaconda3\envs\mimiciv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
D:\software\Anaconda3\envs\mimiciv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [32]:
var_map

,LEVEL2,LEVEL1,ALTERNATIVE,STATUS,STATUS NOTE,ITEMID,MIMIC LABEL,UNITNAME,LINKSTO,COUNT,CATEGORY,CONCEPTID,FLUID,LOINC_CODE,DBSOURCE,Unnamed: 15,PARAM_TYPE,NOTE
0,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,50861,ALANINE AMINOTRANSFERASE (ALT),NaN,labevents,219475.0,CHEMISTRY,NaN,BLOOD,,NaN,NaN,NaN,NaN
1,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,769,ALT,NaN,chartevents,41594.0,Enzymes,NaN,NaN,NaN,carevue,NaN,NaN,NaN
2,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,220644,ALT,None,chartevents,37625.0,Labs,NaN,NaN,NaN,metavision,NaN,Numeric,NaN
3,Albumin,Albumin,NaN,ready,NaN,50862,ALBUMIN,NaN,labevents,146697.0,CHEMISTRY,NaN,BLOOD,1751-7,NaN,NaN,NaN,NaN
4,Albumin,Albumin,NaN,ready,NaN,772,Albumin (>3.2),NaN,chartevents,31022.0,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN
5,Albumin,Albumin,NaN,ready,NaN,1521,Albumin,NaN,chartevents,24454.0,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN
6,Albumin,Albumin,NaN,ready,NaN,227456,Albumin,None,chartevents,22945.0,Labs,NaN,NaN,NaN,metavision,NaN,Numeric with tag,NaN
8,Albumin ascites,Albumin (ascites),NaN,ready,NaN,50835,"ALBUMIN, ASCITES",NaN,labevents,1592.0,CHEMISTRY,NaN,ASCITES,1749-1,NaN,NaN,NaN,NaN
10,Albumin pleural,Albumin (pleural),NaN,ready,NaN,51046,"ALBUMIN, PLEURAL",NaN,labevents,1175.0,CHEMISTRY,NaN,PLEURAL,1748-3,NaN,NaN,NaN,NaN
11,Albumin urine,Albumin (urine),NaN,ready,NaN,51069,"ALBUMIN, URINE",NaN,labevents,2381.0,CHEMISTRY,NaN,URINE,1754-1,NaN,NaN,NaN,NaN


In [54]:
chartitems = var_map.loc[var_map['LINKSTO'] == 'chartevents'].ITEMID

In [55]:
chartitems = chartitems.to_list()

In [67]:
charevent_txt_distanct['is_in_extract'] = charevent_txt_distanct['itemid'].apply(lambda x:1 if x in chartitems else 0)

In [69]:
charevent_txt_distanct.to_csv('charevent.csv', index = False)

In [70]:
ham_id_count = pd.read_sql_query('select count(distinct hadm_id) from mimiciv_derived.icustay_detail',querier.connection ).iloc[0][0]

In [71]:
ham_id_count

66239

In [72]:
query = '''
with t1 as (select itemid, labevents.hadm_id from mimiciv_hosp.labevents inner join mimiciv_derived.icustay_detail on icustay_detail.hadm_id = labevents.hadm_id
where labevents.charttime between icustay_detail.icu_intime and icustay_detail.icu_outtime
group by itemid, labevents.hadm_id)
, t2 as (select itemid, count(*) as count from t1 group by itemid order by count desc)
select  t2.count, d_labitems.* from t2 inner join mimiciv_hosp.d_labitems on t2.itemid = d_labitems.itemid order by count desc;
'''
labevent_with_txt = pd.read_sql_query(query,querier.connection )

In [73]:
labevent_with_txt.insert(1,'percentage', labevent_with_txt['count'] / ham_id_count)

In [74]:
labevent_with_txt

,count,percentage,itemid,label,fluid,category
0,64206,0.969308,50971,Potassium,Blood,Chemistry
1,64206,0.969308,50983,Sodium,Blood,Chemistry
2,64204,0.969278,50902,Chloride,Blood,Chemistry
3,64180,0.968916,50912,Creatinine,Blood,Chemistry
4,64177,0.968870,51006,Urea Nitrogen,Blood,Chemistry
5,64170,0.968765,50882,Bicarbonate,Blood,Chemistry
6,64141,0.968327,50868,Anion Gap,Blood,Chemistry
7,64096,0.967647,51221,Hematocrit,Blood,Hematology
8,64081,0.967421,50931,Glucose,Blood,Chemistry
9,63955,0.965519,51265,Platelet Count,Blood,Hematology


In [75]:
labitems_to_keep = var_map.loc[var_map['LINKSTO'] == 'labevents'].ITEMID

In [76]:
labevent_with_txt = labitems_to_keep.to_list()

In [79]:
labevent_with_txt['is_in_extract'] = labevent_with_txt['itemid'].apply(lambda x:1 if x in labitems_to_keep else 0)

In [80]:
labevent_with_txt

,count,percentage,itemid,label,fluid,category,is_in_extract
0,64206,0.969308,50971,Potassium,Blood,Chemistry,1
1,64206,0.969308,50983,Sodium,Blood,Chemistry,1
2,64204,0.969278,50902,Chloride,Blood,Chemistry,1
3,64180,0.968916,50912,Creatinine,Blood,Chemistry,1
4,64177,0.968870,51006,Urea Nitrogen,Blood,Chemistry,1
5,64170,0.968765,50882,Bicarbonate,Blood,Chemistry,1
6,64141,0.968327,50868,Anion Gap,Blood,Chemistry,1
7,64096,0.967647,51221,Hematocrit,Blood,Hematology,1
8,64081,0.967421,50931,Glucose,Blood,Chemistry,1
9,63955,0.965519,51265,Platelet Count,Blood,Hematology,1


In [82]:
labevent_with_txt.to_csv('labevent.csv')

# save data to csv

In [9]:
query = '''
with t as (select * from mimiciv_derived.chartevent_t1 limit 10000)
select subject_id, hadm_id,stay_id, t.itemid, d_items.label, charttime,lag_time,t.value, t.valuenum,t.valueuom,
       case
        when t.lag_time is not null
        then mimiciv_derived.datetime_diff(t.charttime, t.lag_time, 'SECOND')
        else 0
        end as offset
from t inner join mimiciv_icu.d_items on t.itemid = d_items.itemid
'''
patient_X = pd.read_sql_query(query,querier.connection )

In [10]:
patient_X

,subject_id,hadm_id,stay_id,itemid,label,charttime,lag_time,value,valuenum,valueuom,offset
0,15904173,23836605,30001947,229027,Constitutional,2162-12-26 22:27:00,NaT,Intubated,NaN,None,0.0
1,15904173,23836605,30001947,229027,Constitutional,2162-12-27 02:26:00,2162-12-26 22:27:00,Intubated,NaN,None,14340.0
2,15904173,23836605,30001947,229032,HEENT,2162-12-26 22:27:00,NaT,Oropharynx clear,NaN,None,0.0
3,15904173,23836605,30001947,229032,HEENT,2162-12-26 22:27:00,2162-12-26 22:27:00,Endotracheal tube in place,NaN,None,0.0
4,15904173,23836605,30001947,229032,HEENT,2162-12-26 22:27:00,2162-12-26 22:27:00,PERRL,NaN,None,0.0
5,15904173,23836605,30001947,229032,HEENT,2162-12-26 22:27:00,2162-12-26 22:27:00,Mucous membranes dry,NaN,None,0.0
6,15904173,23836605,30001947,229032,HEENT,2162-12-26 22:27:00,2162-12-26 22:27:00,EOMI,NaN,None,0.0
7,15904173,23836605,30001947,229032,HEENT,2162-12-27 02:26:00,2162-12-26 22:27:00,Endotracheal tube in place,NaN,None,14340.0
8,15904173,23836605,30001947,229032,HEENT,2162-12-27 02:26:00,2162-12-27 02:26:00,PERRL,NaN,None,0.0
9,15904173,23836605,30001947,229032,HEENT,2162-12-27 02:26:00,2162-12-27 02:26:00,Mucous membranes dry,NaN,None,0.0


In [11]:
patient_X.groupby(['subject_id','hadm_id','stay_id','itemid']).apply(lambda x: x.sort_values('charttime', ascending=False))

subject_id   hadm_id   stay_id  \
subject_id hadm_id  stay_id  itemid                                        
10208468   25796414 30002925 220001 7454    10208468  25796414  30002925   
                                    7455    10208468  25796414  30002925   
                             220045 7474    10208468  25796414  30002925   
                                    7473    10208468  25796414  30002925   
                                    7472    10208468  25796414  30002925   
                                    7471    10208468  25796414  30002925   
                                    7470    10208468  25796414  30002925   
                                    7469    10208468  25796414  30002925   
                                    7468    10208468  25796414  30002925   
                                    7467    10208468  25796414  30002925   
                                    7466    10208468  25796414  30002925   
                                    7465    10208468  25796414  30002925   
                                    7464    10208468  25796414  30002925   
                                    7463    10208468  25796414  30002925   
                                    7462    10208468  25796414  30002925   
                                    7461    10208468  25796414  30002925   
                                    7460    10208468  25796414  30002925   
                                    7459    10208468  25796414  30002925   
                                    7458    10208468  25796414  30002925   
                                    7457    10208468  25796414  30002925   
                                    7456    10208468  25796414  30002925   
                             220046 7475    10208468  25796414  30002925   
                             220047 7476    10208468  25796414  30002925   
                             220048 7494    10208468  25796414  30002925   
                                    7493    10208468  25796414  30002925   
                                    7492    10208468  25796414  30002925   
                                    7491    10208468  25796414  30002925   
                                    7490    10208468  25796414  30002925   
                                    7489    10208468  25796414  30002925   
                                    7488    10208468  25796414  30002925   
...                                              ...       ...       ...   
17938576   20818145 30002498 228947 2592    17938576  20818145  30002498   
                                    2591    17938576  20818145  30002498   
                                    2600    17938576  20818145  30002498   
                                    2585    17938576  20818145  30002498   
                                    2581    17938576  20818145  30002498   
                                    2582    17938576  20818145  30002498   
                                    2583    17938576  20818145  30002498   
                                    2584    17938576  20818145  30002498   
                                    2580    17938576  20818145  30002498   
                                    2586    17938576  20818145  30002498   
                                    2587    17938576  20818145  30002498   
                                    2588    17938576  20818145  30002498   
                                    2589    17938576  20818145  30002498   
                                    2579    17938576  20818145  30002498   
                             229090 2626    17938576  20818145  30002498   
                                    2625    17938576  20818145  30002498   
                                    2624    17938576  20818145  30002498   
                                    2623    17938576  20818145  30002498   
                             229091 2630    17938576  20818145  30002498   
                                    2629    17938576  20818145  30002498   
              

In [12]:
patient_X = patient_X.set_index('stay_id')

In [13]:
import os 

In [14]:
for i in set(patient_X.index):
    tmp = patient_X.loc[i].groupby(['subject_id','hadm_id','stay_id','itemid']).apply(lambda x: x.sort_values('charttime', ascending=True))
    tmp_sub = tmp['subject_id'][0]
    tmp_hadm = tmp['hadm_id'][0]
    try:
        os.makedirs("D:/数据/香港实习/MIMICIV/MMICIV_Extract/MIMICIV_Extract/data/test/"+str(tmp_sub)+'/'+str(tmp_hadm))
    except Exception as e:
        print(e)
    tmp.to_csv('D:/数据/香港实习/MIMICIV/MMICIV_Extract/MIMICIV_Extract/data/test/'+str(tmp_sub)+'/'+str(tmp_hadm)+'/'+str(i)+'.csv', index=False)